In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid')

---
# I. Exploratory Data Analysis

In [ ]:
## Loading the data

data_path = "data/"

# Load the data
data = pd.read_csv(data+'de_wiki_terrorism.csv', 
                   header=0, names=['Language', 'Article', 'Date', 'Pageviews'],
                  index_col=0,
                  verbose = True, # Verify if there are any NaNs
                  parse_dates = [2], infer_datetime_format=True)

print("\nBelow the parsed data types :\n", data.dtypes)
print("\nData frame of dimensions", data.shape)
print(f"The dataset includes {data.Article.unique().size} different articles.")
print('\n', data.Article.unique(), '\n')
startdate, enddate = data.Date.min().date(), data.Date.max().date()
period = 12 * (enddate.year - startdate.year) + (enddate.month - startdate.month) +1
print(f"The studied period starts the {startdate} to the {enddate} ({period} months).\n")

data.head()

In [ ]:
#Understanding the index
print("Are the values in the first column unique ? ", data.index.is_unique)
print("If unique, indicates the first column can be used as an index.\n")

# Sort by index to get articles per date per period
data.sort_index(axis = 0, na_position='first', inplace=True)

data.head()

In [ ]:
# Language column contains only one value and can be dropped.
print(f"Language column contains {data.Language.unique().size} value(s) : {data.Language.unique()}")
print("It can thus be removed.\n")

data.drop(labels='Language', axis=1, inplace=True)
data.head()

## A. Cleaning and handling missing/undefined values
Undefined values are any non-positive numerical or non-numerical values. In case of missing values, we will discuss the possible fixes and consequently decide on our way to handle them.

In [ ]:
## probably nothing TODO

## B. Consistency in Datasets
Here we will verify that all days were correctly retrieved.

In [ ]:
## TODO : check that no day is missing on the period

---
# II. Generic Methods

## A. Monthly Aggregation
The data in our reference paper $\text{Chilling Effects: Online Surveillance and Wikipedia Use}$ (*Jonathan W. Penney*), data is aggregated on a monthly basis and further computations are done on this transformation.

In [3]:
def agg_monthly(df) :
    """
    Aggregates the pageviews in a monthly fashion. Returns a DataFrame with only the numerical columns summed.
    Note : Sets the index to date.
    
    Parameters 
    ----------
    df : pd DataFrame
        df should have at least 3 columns containing `Article`, `Date` and `Pageviews`. Column with `Date` should be named 'Date'.
    """
    # Aggregate monthly
    dataMonthly = df.set_index('Date')
    dataMonthly = dataMonthly.resample('M').sum()
    # Verify period
    print(f"The formatted dataset correctly spans from {dataMonthly.index.min().date()} to {dataMonthly.index.max().date()}.\n")
    
    return dataMonthly

## B. Mean Computation
The first analysis technique done in the paper is a comparison of means before and after the interruption event. Let us write the methods for this purpose given a monthly pageviews dataset.

In [ ]:
## TODO

## C. ITS Regression 
Following the formula of segmented regression of an interrupted time series (ITS) : 
$$
Y_{t} = \beta_{0}*\text{time}+\beta_{2}*\text{intervention}+\beta_{3}*\text{postslope}+\epsilon_{1}
$$
We define in the following the methods to perform such a regression.

In [ ]:
## TODO 
## @Gonxhe si tu peux mettre ton code c'est cool!

## D. Plotting
The reference paper uses 3 types of plots to describe the results :
* **Histogram** of means before and after the interruption event (*cf. figure 1 in reference paper*).
* **Segmented regression** around the interrupted event for **1 set** of articles (*cf. figure 2 in reference paper*).
* **Segmented regression** around the interrupted event for **2 sets** of articles (*cf. figure 4A in reference paper*).

Additionally, the regression plots can feature confidence intervals if prompted by the user (cf. figure 4 in reference paper).

In [ ]:
## TODO plot a histogram of means (cf. Fig 1)

In [21]:
def segmented_reg_1_set(df, interruption, verbose=False, ytick_step=5e5, narts=None) :
    """
    Plots the ITS of a dataset of monthly Pageviews around an interruption date.
    
    Parameters 
    ----------
    df : pd DataFrame
        Pandas DataFrame containing the monthly pageviews around the interruption date.
    interruption : string
        Interruption date in the form of a string. Please use following fashion :
        1st of May 1997 --> 1995-05-01
    verbose : bool (default: False)
        Display temp dataframes created in the process and other insightful information from the process.
    ytick_step : int (default: 5e5)
        Step in pageviews between 2 ticks on the y axis
    narts : int (default: None)
        Number of articles summed in the monthly Pageviews. Used for the legend.
    """
    # Dividing dataset and excluding the revelations date
    revelations = pd.to_datetime(interruption)

    # Creating both datasets
    pre = df.loc[df.index < revelations]
    post = df.loc[df.index > revelations]
    if verbose :
        print("Pre-interruption pageviews:\n", pre.head())
        print("\nPost-interruption pageviews:\n", post.head())

    # Verifying operation
    if verbose : 
        print("Verifying coherence for split around interruption date...")
    assert(df.size == (pre.size + post.size))
    if verbose :
        print(df.size == (pre.size + post.size))
        
    ##########################################################
    # Plotting
    
    # Create dataframe with both periods identified
    concatenated = pd.concat([
        pre.reset_index(drop=True).assign(period='pre'), 
        post.reset_index(drop=True).assign(period='post')], ignore_index=True)

    # Creating a month-identifier column
    concatenated = concatenated.reset_index()
    concatenated.columns.values[0]='Months'
    concatenated['Months'] = concatenated['Months']+1
    # The paper shifts monthly count to start from 1.

    # Identifying both periods visually
    pal = dict(pre="black", post="grey")
    g = sns.FacetGrid(concatenated, hue='period', palette=pal, height=5, aspect=1.5, xlim=(-1, 33))

    #### TODO use computed regression !
    # Creating the regressions for both periods
    g.map(sns.regplot, "Months", "Pageviews", ci=None, robust=1, scatter=False)

    # Adding the scatter data separately
    g.map(sns.scatterplot,  "Months", "Pageviews", color="black")

    # Configuring the axes to fit the data
    plt.xticks(np.arange(0, concatenated.shape[0]+2, 2), np.arange(0, concatenated.shape[0]+2, 2, dtype=int))
    ymin, ymax = floor(concatenated['Pageviews'].min()/ytick_step)*ytick_step, floor(concatenated['Pageviews'].max()/ytick_step +1)*ytick_step
    plt.yticks(np.arange(ymin, ymax, ytick_step), np.arange(ymin, ymax, ytick_step, dtype=int))

    # Configuring the labels to be as on paper
    plt.ylabel("Total Views"+ ( '('+narts+' Wikiğedia Articles)' if narts else ''))
    plt.xlabel("Time (Months)")

    # Adding grid to be as on paper
    plt.grid(True, which='major', axis='y', alpha=0.3)

    # Creating legend
    rev_date=revelations.month_name()+' '+revelations.year
    legend = plt.legend(labels=["Trend Pre-"+rev_date, "Trend Post-"+rev_date, "Total Article Views (per month)"], 
                        bbox_to_anchor=[1,-0.1], fontsize='medium', 
                        ncol=2, borderpad=0.75, handlelength=5)

    # Adding June 2013 line identifier
    plt.text(17.5,4.2e6,'Mid '+rev_date,horizontalalignment='center')
    plt.axvline(17.5, linewidth=2.5, color='black')

    plt.show()
    
    return None

In [22]:
## TODO Compute plotting for 2 sets 

---
# III. GDPR Adoption
The European Parliament adopted the GDPR Regulation text on the $27^{th}$ of April $2016$. Our first case of interest are the following research questions at this date : 
* Is there an inverse chilling effect (e.g. immediate spike) following the announcement of the regulation adoption?
* Has there been a change in trends in the viewing of critical terms related to the regulation after the event?

We will thus answer these questions using the analysis methods from the reference paper and draw conclusions.

## A. Monthly Aggregation

## B. Mean Computation

## C. ITS Regression

---
# GPDR Begginig of enforcement
The GDPR regulation was applied starting from the $25^{th}$ of May $2018$ for every country of the European Union (EU) and European Economic Area (EEA). We aim to answer the following questions regarding the event : 
* Is there an inverse chilling effect (e.g. immediate spike) following the day when the regulation became effective?
* Has there been a change in trends in the viewing of critical terms related to the regulation after the event?

We will thus answer these questions using the analysis methods from the reference paper and draw conclusions.

## A. Monthly Aggregation

## B. Mean Computation

## C. ITS Regression

# Conclusion